# Import packages

In [ ]:
import copy
import sys
sys.path.insert(0, "../src")

from datahandler.datahandler import DataHandler
from pointcloud.pointcloud import PointCloudManip
from segmentation.segmentation import Segmentation
import utils.utils as utils

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

# Set variables

In [ ]:
root = "../data/data_10_31"
img_path = "capture-31-10-1"
calib_path = "calibration_2022_10_31_14_59_14_1667224754485089818"

cams_to_keep = None  # Can be None or ["215122254778", "151422251878", "215122256544", "213522251068"]

In [ ]:
zoom = 1.0
front = [0.0, 0.0, 1.0]
up = [1.0, 0.0, 0.0]

coord = o3d.geometry.TriangleMesh.create_coordinate_frame(0.5, [0, 0, 0])

# Load data

In [ ]:
dh_ = DataHandler(root, img_path, calib_path, cams_to_keep)

In [ ]:
person_rgb, person_depth = dh_.load_images("mans")
bg_rgb, bg_depth = dh_.load_images("background")

trans = dh_.load_transformations()
depth_scales = dh_.load_depth_scales()
intrinsics = dh_.load_intrinsics()

In [ ]:
print(depth_scales)

In [ ]:
print(trans)

In [ ]:
print(intrinsics)

In [ ]:
utils.plot_rgbd(list(person_rgb.values()), list(person_depth.values()))

In [ ]:
utils.plot_rgbd(list(bg_rgb.values()), list(bg_depth.values()))

# Segment person

In [ ]:
seg = Segmentation()

In [ ]:
masks = seg.get_person_masks(list(person_rgb.values()))

In [ ]:
out_rgb, out_dep = seg.apply_masks(list(person_rgb.values()), masks, list(person_depth.values()))

In [ ]:
utils.plot_rgbd(out_rgb, out_dep)

# Depth filtering

In [ ]:
threshold = 1.7

_, scale = list(depth_scales.items())[0]
thr = threshold / scale
out_dep_filt = [np.where(im <= thr, im, 0) for im in out_dep]

In [ ]:
utils.plot_rgbd(out_rgb, out_dep_filt)

# Create Point Cloud

In [ ]:
pcd = PointCloudManip.create_point_cloud(
    (
        dict(zip(person_rgb.keys(), out_rgb)),
        dict(zip(person_depth.keys(), out_dep_filt)),
    ),
    depth_scales,
    trans,
    intrinsics
)

In [ ]:
o3d.visualization.draw_geometries(
    [pcd, coord], zoom=zoom, front=front, lookat=pcd.get_center(), up=up
)

### Convex hull

In [ ]:
hull, _ = pcd.compute_convex_hull()

o3d.visualization.draw_geometries(
    [hull, coord], zoom=zoom, front=front, lookat=hull.get_center(), up=up
)

In [ ]:
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))

o3d.visualization.draw_geometries(
    [pcd, hull_ls, coord], zoom=zoom, front=front, lookat=pcd.get_center(), up=up
)

### Distance filtering

In [ ]:
pcd_person = PointCloudManip.create_point_cloud(
    (person_rgb, person_depth), depth_scales, trans, intrinsics
)

In [ ]:
pcd_bg = PointCloudManip.create_point_cloud((bg_rgb, bg_depth), depth_scales, trans, intrinsics)

In [ ]:
pcd_f = PointCloudManip.distance_filter(pcd_person, pcd_bg, 0.06)


In [ ]:
o3d.visualization.draw_geometries(
    [pcd_f, coord], zoom=zoom, front=front, lookat=pcd_f.get_center(), up=up
)

### Segment out plane

In [ ]:
pcd_s = PointCloudManip.segment_out_plane(pcd_person, 0.05)

In [ ]:
o3d.visualization.draw_geometries(
    [pcd_s, coord], zoom=zoom, front=front, lookat=pcd_s.get_center(), up=up
)

### Simple Crop

In [ ]:
print(f"Min bounds: {pcd_s.get_min_bound()}")
print(f"Max bounds: {pcd_s.get_max_bound()}")

In [ ]:
min_bounds = (-1.0, -2.0, 0.0)
max_bounds = (1.0, 2.0, 2.0)
pcd_c = PointCloudManip.crop_pcd(pcd_s, min_bounds, max_bounds)

In [ ]:
o3d.visualization.draw_geometries(
    [pcd_c, coord], zoom=zoom, front=front, lookat=pcd_c.get_center(), up=up
)

# Create Mesh

In [ ]:
mesh = utils.create_mesh_tsdf(
    dict(zip(person_rgb.keys(), out_rgb)),
    dict(zip(person_depth.keys(), out_dep_filt)),
    depth_scales,
    trans,
    intrinsics
)

In [ ]:
o3d.visualization.draw_geometries(
    [mesh, coord], zoom=zoom, front=front, lookat=mesh.get_center(), up=up
)

## RGBD from Point cloud

In [ ]:
rot_pcd = PointCloudManip.rotate_pcd(pcd, (-np.pi / 2, 0, 0))

In [ ]:
o3d.visualization.draw_geometries([rot_pcd, coord])

In [ ]:
rgb, dep = PointCloudManip.pcd_to_rgbd(
    rot_pcd, intrinsics["215122254778"], depth_scales["215122254778"]
)

In [ ]:
utils.plot_rgbd([rgb], [dep])